**This is the codebook about 1 VS N labelled classification.
Before modeling, we preprocess the data, which included drop unimportant risk types and drop duplicated news.
The methods we used: Logistics, new feature(word count) and oversample.    
Finally, the logitics without word count perform best, which obtain acc 0.86 and F1 score (0.91 and 0.66).**

In [1]:
import pickle
# open a file, where you stored the pickled data
file = open('traintest_2019NOV_RS_UNITED.pkl', 'rb')

# dump information to that file
data = pickle.load(file)

# close the file
file.close()

In [2]:
data['risk_cat'].nunique()

19

In [3]:
data.groupby('risk_cat')['contents'].count().sort_values(ascending=False)

risk_cat
Not Risk                        1271
Others                           927
Employee Misconduct              763
Service Failure/Availability     746
Changes in Company Policy        422
Product Availability             330
Product Failure                  244
Macro Environment                196
Industry Adjacent                153
Invalid                          134
Customer Misconduct               92
Confrontation                     91
Organizational Reform             90
Financial                         60
Media Blunder                     39
Auxiliary                         37
Natural Disaster                  35
Technological Infrastructure      13
Malevolence                        1
Name: contents, dtype: int64

**From the aboved results, there are 19 risk type in totall, we need to further filter the types.  
After grouping and count the amount of data in each type.  
We decde to drop malevolence.**

In [24]:
df = data.loc[data['risk_cat']!='Malevolence']

**In case of some news have duplicated contents. We decide to check and delete duplicate content.**

In [17]:
import re
import time

from matplotlib import pyplot as plt
from simhash import Simhash, SimhashIndex

import warnings
warnings.filterwarnings("ignore")

In [29]:
def check_duplicates(index, objs):
    dups_ = []
    for i in range(len(objs)):
        dups = index.get_near_dups(objs[i][1])
        if len(dups) > 1:
            dups.sort()
            dups_.append(tuple(dups))
    return list(set(dups_))

def deep_unique(li):
    using_ = li.copy()
    for i in range(len(li)):
        cur_tup = li[i]
        l = len(cur_tup)
        for j in li[i+1:]:
            for k in cur_tup:
                if k in j and l < len(j):
                    if cur_tup in using_:
                        using_.remove(cur_tup)
                    break
    return using_

def check_label(i1, i2):
    return df['risk_cat'].iloc[i1] == df['risk_cat'].iloc[i2]

content = df.contents.to_numpy()
objs = []
for k in range(len(content)):
    objs.append((str(k), Simhash(content[k])))
    
index = SimhashIndex(objs, k=3)
dups = check_duplicates(index, objs)
dups_unique = deep_unique(dups)

consistent = []
inconsistent = []

for dups in dups_unique:
    flag = True
    for j in range(len(dups)):
        if j + 1 < len(dups) and not check_label(int(dups[j]),int(dups[j+1])):
            flag = False
            break
    if flag:
        consistent.append(dups)
    else:
        inconsistent.append(dups)

print(f"There are {len(dups_unique)} duplicated list of articles.")
print(f"Out of them, {len(consistent)} list of articles has consistent labels while {len(inconsistent)} does not.")

from collections import Counter
flat_inconsistent = []
revise_labels = {}
for item in inconsistent:
    if len(item)==2:
        flat_inconsistent.extend(list(item))
    else:
        labels = [df['risk_cat'].iloc[int(i)] for i in item ]
        temp = Counter(labels).most_common()[0]
        if temp[1]/len(labels)>0.5:
            for i in item:
                revise_labels[i] = temp[0]
        else:
            flat_inconsistent.extend(list(item)) 
flat_inconsistent = [ item  for item in flat_inconsistent if int(item) not in revise_labels]
remove_inconsistent = [item  for item in df.index if str(item) not in flat_inconsistent]

df_prep1 = df.loc[remove_inconsistent,:]
print("%s inconsistent articles are removed"%len(flat_inconsistent))

#flat_inconsistent = [ item  for item in flat_inconsistent if item not in revise_labels]
#remove_inconsistent = [item  for item in df.index if str(item) not in flat_inconsistent]

#df_prep1 = df.iloc[remove_inconsistent,:]
#print("%s inconsistent articles are removed"%len(flat_inconsistent))

#for key,item in revise_labels.items():
   # df_prep1.loc[int(key)]['risk_cat'] = item
#print("After remove and relabel risk category, there are %s news" %len(df_prep1))

There are 276 duplicated list of articles.
Out of them, 157 list of articles has consistent labels while 119 does not.
156 inconsistent articles are removed


In [31]:
df_prep1 = df_prep1.reset_index()

In [33]:
df_prep1.groupby('risk_cat')['contents'].count().sort_values(ascending=False)

risk_cat
Not Risk                        1244
Others                           893
Employee Misconduct              737
Service Failure/Availability     730
Changes in Company Policy        412
Product Availability             323
Product Failure                  243
Macro Environment                188
Industry Adjacent                149
Invalid                          133
Customer Misconduct               92
Confrontation                     90
Organizational Reform             84
Financial                         59
Media Blunder                     38
Auxiliary                         35
Natural Disaster                  34
Technological Infrastructure      13
Name: contents, dtype: int64

In [44]:
1244+893+737+730+412+323+243+188+149+133+92+90

5234

In [46]:
5234/data.shape[0]

0.9273564847625797

**We do not want to keep all categories, so we only remain majority categories, aka top 12 categories.**

In [52]:
df_prep2 = df_prep1.loc[df_prep1['risk_cat'].isin(['Macro Environment', 'Product Availability','Not Risk', 
                                                 'Industry Adjacent','Invalid',  'Product Failure',
                                                'Service Failure/Availability', 'Others', 'Changes in Company Policy',
                                                 'Employee Misconduct','Customer Misconduct', 'Confrontation']) ]

In [55]:
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import unicodedata

def tokenize_lemmatize(doc):
    doc = re.sub("[\xa0]+","",doc)
    words = [v.lower() for v in word_tokenize(doc) if v.isalpha() or v.isdigit()]
    words = [w for w  in words if w not in stopWords] 
    words = [''.join((c for c in unicodedata.normalize('NFD', w) if unicodedata.category(c) != 'Mn')) for w in words]
    lemmatizer = WordNetLemmatizer() 
    words = [lemmatizer.lemmatize(w) for w in words]
    words = ["#number" if w.isdigit() else w for w in words]
    return words

In [56]:
content = df_prep2['contents'].values
content = [tokenize_lemmatize(doc) for doc in content]

In [57]:
org_vacab  = Counter([v for item in content for v in item]).most_common() 
print("The orignal vacabuary size is %s" %len(org_vacab))
vocab_ = [item_value[0] for item_value in org_vacab if item_value[1]>5]
print("The refined vacabuary size is %s" %len(vocab_))
content = [" ".join([v for v in item if v in vocab_]) for item in content ]

The orignal vacabuary size is 32380
The refined vacabuary size is 11072


In [58]:
df_prep2 = df_prep2.drop(columns=['level_0','index'])

In [60]:
df_prep2['contents'] = content

In [106]:
df_prep2.to_csv('df_prep2.csv')

In [4]:
from sklearn import model_selection, preprocessing, linear_model, metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [1]:
import pandas as pd
import numpy as np

In [8]:
df_prep3 = df_prep2.copy()
l1 = []
for i in range(df_prep3.shape[0]):
    if df_prep3['risk_cat'][i]=='Not Risk':
        l1.append(1)
    else:
        l1.append(0)
df_prep3['Not Risk'] = l1

In [9]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(df_prep3.contents,df_prep3['Not Risk'],
                                                                    test_size = 0.2, random_state = 42) 
tfidf1 = TfidfVectorizer() 
tfidf1.fit(x_train)  
x_train= tfidf1.transform(x_train) 
x_test = tfidf1.transform(x_test) 
df_prep3['word_count'] = [len(item.split(" ")) for item in df_prep3['contents']]
x_train_wc =  hstack([x_train,np.array([df_prep3.iloc[y_train.index.values]['word_count']]).T])
x_test_wc =  hstack([x_test,np.array([df_prep3.iloc[y_test.index.values]['word_count']]).T])

In [10]:
lr1.fit(x_train_wc, y_train)
y_pred_log = lr1.predict_proba(x_test_wc)


y_pred_log = [np.argmax(line) for line in y_pred_log]
print('accuracy %s' % accuracy_score(y_pred_log, y_test))
print(classification_report(y_test, y_pred_log))

accuracy 0.8605539637058262
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       803
           1       0.76      0.58      0.66       244

    accuracy                           0.86      1047
   macro avg       0.82      0.76      0.79      1047
weighted avg       0.85      0.86      0.85      1047



C:\Users\E055761\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


OVR classification   
without wordcount + logistics

In [12]:

lr2 = LogisticRegression(random_state=123,penalty='l2', solver='lbfgs')
lr2.fit(x_train, y_train)
y_pred_log = lr2.predict_proba(x_test)


y_pred_log = [np.argmax(line) for line in y_pred_log]
print('accuracy %s' % accuracy_score(y_pred_log, y_test))
print(classification_report(y_test, y_pred_log))

accuracy 0.8643744030563515
              precision    recall  f1-score   support

           0       0.88      0.96      0.92       803
           1       0.80      0.56      0.66       244

    accuracy                           0.86      1047
   macro avg       0.84      0.76      0.79      1047
weighted avg       0.86      0.86      0.86      1047



OVR classification   
oversample + wordcount + log

In [16]:
#oversample wordcount
from imblearn.over_sampling import RandomOverSampler
os = RandomOverSampler(sampling_strategy='minority')
x_train_wc_os, y_train_os = os.fit_sample(x_train_wc,y_train)


lr3 = LogisticRegression(random_state=123,penalty='l2', solver='lbfgs')
lr3.fit(x_train_wc_os, y_train_os)
y_pred_log = lr3.predict_proba(x_test_wc)


y_pred_log = [np.argmax(line) for line in y_pred_log]
print('accuracy %s' % accuracy_score(y_pred_log, y_test))
print(classification_report(y_test, y_pred_log))


accuracy 0.828080229226361
              precision    recall  f1-score   support

           0       0.92      0.85      0.88       803
           1       0.61      0.75      0.67       244

    accuracy                           0.83      1047
   macro avg       0.76      0.80      0.78      1047
weighted avg       0.85      0.83      0.83      1047



C:\Users\E055761\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


OVR classification   
oversample + without word count + log

In [17]:
#oversample withoutcount
os = RandomOverSampler(sampling_strategy='minority')
x_train_os, y_train_os = os.fit_sample(x_train,y_train)


lr4 = LogisticRegression(random_state=123,penalty='l2', solver='lbfgs')
lr4.fit(x_train_os, y_train_os)
y_pred_log = lr4.predict_proba(x_test)


y_pred_log = [np.argmax(line) for line in y_pred_log]
print('accuracy %s' % accuracy_score(y_pred_log, y_test))
print(classification_report(y_test, y_pred_log))

accuracy 0.8271251193887297
              precision    recall  f1-score   support

           0       0.92      0.85      0.88       803
           1       0.60      0.75      0.67       244

    accuracy                           0.83      1047
   macro avg       0.76      0.80      0.78      1047
weighted avg       0.84      0.83      0.83      1047

